# Experimental DSS Algorithm
## Radar Protection Without a Centralized Spectrum Access Administrator
### 19 August 2024

This routine simulates a closed-loop interference mitigation approach whereby:<br>
1. The location of the protected Federal system is known to the Commercial user.
2. The Federal system publishes its target interference level and monitors aggregate interference.
3. Commercial users estimate path loss from their transmitters to the Federal System using an authorized propagation model.
4. Based on _actual_ received interference, the Federal system adjusts the target protection threshold,<br>
either to allow an increase or to require a decrease in Commercial emissions.
5. Commercial devices modify their transmit powers accordingly, albeit without knowing the aggregate interference.
6. The process iterates until the Federal system detects interference just below the allowable threshold.

## Set Input Parameters
### Radar Protection Threshold
In the window below, set the intial threshold for radar protection.<br>
This value represents the amount of aggregate interference, referenced to the radar antenna, which would cause unacceptable impairment.
### Define Commercial Transmitter Attributes
Dimension and describe the commercial transmitters that will contribute to aggregate interference.<br>
- Enter the total number of commercial base stations.
- Choose the path loss associated with the strongest interferer (Lowest Path Loss).
- Specify the difference in path loss among the commercial base stations (Path Loss Delta).
- Specify the power spectral density, bandwidth, and antenna gain associated with each base station.

Notes:
1. This model allows two types of radios to be specified, with two different transmit power capabilities.<br>
For an array of transmitters, the model will assign an equivalent number of each type starting with the low power type.
2. In determining aggregate interference, the model assigns the lowest path loss to the first two transmitters.<br>
Then, for each successive pair of transmitters, link loss is increased by the Path Loss Delta.
3. The maximum radiated PSD is 37 dBm/MHz.
#### When finished, click the _Capture Settings_ button.

In [4]:
# Import Interactive Libraries
import ipywidgets as wg
from IPython.display import display, HTML

# Style and Layout Attributes
styl = {'description_width':'initial'}
box = wg.Layout(width='250px')

#
set_state = wg.HTML(value="<i><font color='red'>Click to store input parameters</font></i>")

# Intialize the Radar Protection Threshold
styl = {'description_width':'initial'}
slider_long = wg.Layout(width='450px')
init_Thresh = wg.IntSlider(description='Radar Protection Threshold (dBm/MHz):',value=-144,min=-154,max=-134,step=1,style=styl,layout=slider_long)

# Number of Base Stations and Lowest Path Loss
num_Tx = wg.BoundedIntText(description='Number of Base Stations:',value=10,min=1,max=30,step=1,style=styl)
init_PL = wg.IntSlider(description='Lowest Path Loss (dB):',value=-170,min=-200,max=-150,step=5,style=styl)
delta_PL = wg.BoundedIntText(description='Path Loss Delta Among Links (dB):',value=5,min=0,max=10,step=1,style=styl)

# Transmitter Attributes: Power, Gain, Bandwidth
offset = wg.BoundedIntText(description='Transmitter Power Difference (dB):',value=3,min=0,max=10,step=1,style=styl,layout=box)
txPSD_low = wg.IntSlider(description='Low Power Tx (dBm/MHz):',value=26,min=19-offset.value,max=37-offset.value,step=1,style=styl)
txPSD_high = wg.IntSlider(description='High Power Tx (dBm/MHz):',value=29,min=19,max=37,step=1,style=styl)
ant_Gain = wg.BoundedIntText(description='Transmit Antenna Gain (dB):',value=11,min=0,max=8,step=1,style=styl)
tx_BW = wg.BoundedIntText(description='Operating Bandwidth (MHz):',value=20,min=10,max=80,step=10,style=styl)

# Functions that will keep releated parameters in sync
def update_high_from_low(change):
    txPSD_high.value = change['new'] + offset.value
def update_low_from_high(change):
    txPSD_low.value = change['new'] - offset.value
def update_both_from_offset(change):
    txPSD_high.value = txPSD_low.value + change['new']
    txPSD_low.min = 19-change['new']
    txPSD_low.max = 37-change['new']
def update_gain_from_highPSD(change):
    ant_Gain.max = 37-txPSD_high.value
def reset_state(change):
    set_state.value = "<i><font color='red'>Click to capture new settings</font></i>"

# Automatically adjust linked parameters when one is changed
txPSD_low.observe(update_high_from_low,names='value')
txPSD_high.observe(update_low_from_high,names='value')
offset.observe(update_both_from_offset,names='value')
txPSD_high.observe(update_gain_from_highPSD,names='value')

# Reset status upon any parameter change
init_Thresh.observe(reset_state,names='value')
num_Tx.observe(reset_state,names='value')
init_PL.observe(reset_state,names='value')
delta_PL.observe(reset_state,names='value')
offset.observe(reset_state,names='value')
txPSD_low.observe(reset_state,names='value')
txPSD_high.observe(reset_state,names='value')
ant_Gain.observe(reset_state,names='value')
tx_BW.observe(reset_state,names='value')

radar_Inputs={
    'cur_InitThresh' : init_Thresh.value
}
tx_Inputs={
    'cur_NumTx' : num_Tx.value,
    'cur_InitPL' : init_PL.value,
    'cur_DeltaPL' : delta_PL.value,
    'cur_TxPSDlow' : txPSD_low.value,
    'cur_TxPSDhigh' : txPSD_high.value,
    'cur_Offset' : offset.value,
    'cur_AntGain' : ant_Gain.value,
    'cur_TxBW' : tx_BW.value
}

# Create Update Button
update_Button = wg.Button(description="Capture Settings")
current_Values = wg.Output()

def update_settings(b):
    with current_Values:
        current_Values.clear_output()
        radar_Inputs['cur_InitThresh'] = init_Thresh.value
        tx_Inputs['cur_NumTx'] = num_Tx.value
        tx_Inputs['cur_InitPL'] = init_PL.value
        tx_Inputs['cur_DeltaPL'] = delta_PL.value
        tx_Inputs['cur_TxPSDlow'] = txPSD_low.value
        tx_Inputs['cur_TxPSDhigh'] = txPSD_high.value
        tx_Inputs['cur_Offset'] = offset.value
        tx_Inputs['cur_AntGain'] = ant_Gain.value
        tx_Inputs['cur_TxBW'] = tx_BW.value
        set_state.value = "<font color='green'>Values updated successfully</font>"
        #print(f"Current Protection Threshold: {radar_Inputs}")
        #print(f"Current Transmitter Settings: {tx_Inputs}")

update_Button.on_click(update_settings)

# Layout of Widgets
grid1 = wg.GridspecLayout(7,3)
grid1[0,:] = init_Thresh
grid1[1,0] = wg.HTML(value="--------------------------------------------------------")
grid1[1,1] = wg.HTML(value="--------------------------------------------------------")
grid1[1,2] = wg.HTML(value="--------------------------------------------------------")
grid1[2,0] = num_Tx
grid1[2,1] = init_PL
grid1[2,2] = delta_PL
grid1[3,0] = wg.HTML(value="--------------------------------------------------------")
grid1[3,1] = wg.HTML(value="--------------------------------------------------------")
grid1[3,2] = wg.HTML(value="--------------------------------------------------------")
grid1[4,0] = txPSD_low
grid1[5,0] = txPSD_high
grid1[6,0] = offset
grid1[4,1] = ant_Gain
grid1[5,1] = tx_BW
grid1[4,2] = update_Button
grid1[5,2] = set_state
display(grid1)

GridspecLayout(children=(IntSlider(value=-144, description='Radar Protection Threshold (dBm/MHz):', layout=Lay…

## Compute and Mitigate Interference
* Click the _Calculate Interference_ button to evaluate interference at the radar based on the transmitter attributes specified above.
* The output will illustrate aggregate interference, amount of attenutaion needed, and the contribution from each transmitter.
* After reviewing the output, the nominal interference level or the radar protection threshold can be adjusted.<br>Simply modify the settings above and then click _Capture Settings_.
* After making a change, click _Calculate Interference_ again.
* Otherwise, select the preferred interference mititation algorithm and click _Run DSS Algorithm_.

Available Algorithms:
1. _Converge to the Same Level_:   Strong interferers decrease their EIRP, while weak interferers are permitted to increase EIRP until the target protection threshold is reached.  The net result is a tendency for the received interference from all links to converge toward the same value.
2. _Disallow Power Increases_:   Similar to the first method, except that no transmitter is permitted to increase its EIRP.  The net result is that some of the stronger interferers would require less attenuation to meet the target protection threshold.
3. _Disable After Tx Threshold_:   This method assumes that operators would prefer to disable a transmitter if the attenuation required exceeds a certain threshold.  The net result is that disabled transmitters no longer contribute to the interference observed at the Federal system, and the target protection threshold can be met with less attenuation among the remaining active transmitters.

#### When finished, scroll down to review the outcomes of each iteration and the adjustments for each transmitter.

In [5]:
### Import Libraries
import math as m
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class InterferenceMitigation:

    #Initialize
    def __init__(self):
        # Initialize State Variables and Output Widgets
        self.thresh_Radar = None
        self.num_Transmitters = None
        self.tx_RadPSD = None
        self.tx_MaxPSD = None
        self.tx_EIRP = None
        self.path_Loss = None
        self.rx_RSSI = None
        self.rx_PSDdB = None
        self.rx_PSDmW = None
        self.int_AggmW = None
        self.int_AggdB = None
        self.tx_Labels = None
        self.pred_err = None
        self.algo_Option = 1
        self.algo_Options = ['Converge to Same Rx Level',
                             'Disallow Tx Power Increases',
                             'Disable Tx After Threshold',
                             '*** Compare All ***']
        self.calcs = wg.Output()
        self.algo = wg.Output()
                    
        # Create Interference Calculation Widgets
        styl = {'description_width':'initial'}
        dd_long = wg.Layout(width='360px')
        self.calc_interf_button = wg.Button(description="Calculate Interference")
        self.run_dss_button = wg.Button(description="Run DSS Algorithm")
        # self.compare_button = wg.Button(description="Compare Algorithms")
        self.pred_err_box = wg.Checkbox(
            description="Apply Prediction Error (dB)",
            value=False,disabled=False,indent=False,
            layout=wg.Layout(width='auto')
        )
        self.pred_err_value = wg.BoundedIntText(
            value=6,min=1,max=12,step=1,
            layout=wg.Layout(width='45px')
        )
        self.algo_selection = wg.Dropdown(
            description='Select DSS Algorithm >>',
            options=self.algo_Options,
            value=self.algo_Options[0],
            disabled=False,
            style=styl,
            layout=dd_long
        )
        self.tx_dis_thresh = wg.BoundedIntText(
            description='Maximum Attenuation to Tolerate (dB) >>>',
            value=-10,min=-20,max=-3,step=1,
            layout=dd_long,style=styl
        )
        self.pred_err_placeholder = wg.HTML("")
        self.tx_dis_placeholder = wg.HTML("")
        self.stack1 = wg.Stack([self.pred_err_placeholder, self.pred_err_value])
        self.pred_err_box.observe(self.on_pred_err_change, names='value')
        self.stack2 = wg.Stack([self.tx_dis_placeholder, self.tx_dis_placeholder, self.tx_dis_thresh,self.tx_dis_thresh], selected_index=0)
        wg.jslink((self.algo_selection, 'index'),(self.stack2, 'selected_index'))
              
        # Monitor Prediction Error Checkbox and Algorithm Selection for Changes
        self.pred_err_box.observe(self.on_pred_err_change, names='value')
        self.algo_selection.observe(self.on_algo_change, names='value')
        
        # Register Callbacks for Buttons
        self.calc_interf_button.on_click(self.calc_clicked)
        self.run_dss_button.on_click(self.run_clicked)
        # self.compare_button.on_click(self.compare_clicked)
        
        # Display Widgets
        self.calc_interf_button.layout.margin = '0 90px 0 0'  # right margin of 50px
        # calc_box = wg.HBox([self.calc_interf_button,wg.Layout(width='20px')])
        # err_val = wg.HBox([self.stack1,wg.HTML("(dB)")])
        err_box = wg.HBox([self.pred_err_box,self.stack1])
        col_1 = wg.VBox([self.calc_interf_button,err_box])
        col_2 = wg.VBox([self.algo_selection,self.stack2])
        col_3 = wg.VBox([self.run_dss_button])
        # col_4 = wg.VBox([self.compare_button])
        controls = wg.HBox([col_1,col_2,col_3])
        display(controls)
        
        # Display Outputs
        display(self.calcs)
        display(self.algo)

    # Define a Function to Handle Changes in Prediction Error Checkbox
    def on_pred_err_change(self,change):
        with self.algo:
            self.algo.clear_output()  # clear previous inteference info
            self.stack1.selected_index = 1 if self.pred_err_box.value else 0
            
    # Define a Function to Handle Changes in Alorithm Selection
    def on_algo_change(self,change):
        with self.algo:
            selected_value = change['new']
            self.algo_Option = self.algo_Options.index(selected_value)+1
              
    # Describe Functions for Interference Calculation and DSS Algorithm
    # Generate Path Loss Estimation Errors and Create Array of Real Path Loss Values
    def real_path_loss(self, nIter, array_pl, mask_tx, err_mean, err_stddev):
        if self.algo_Option == 4:
            err_row = self.arr_Random[nIter]
            errs = err_row[mask_tx]
        else:
            errs = np.random.normal(loc=err_mean, scale=err_stddev, size=len(array_pl))
        return array_pl + errs

    # DSS Algorithm 1: Converge to same received level
    def dss_converge_rx(self, nIt, thresh_Target, gain, pathLoss, thresh_Adj, interf, tx_PSD, txPSD_Table):
        n=0
        # Adjust Transmit Powers to Meet Target Inteference Level
        while interf[n] > thresh_Target or (interf[n] < thresh_Target-1 and not np.all(37==tx_PSD)):
        # while (interf[n] > thresh_Target or interf[n] < thresh_Target-1) and not np.all(37==tx_PSD): # Rauf's original conditional
            for k in range(self.num_Transmitters):
                if tx_PSD[k]+pathLoss[k] > thresh_Adj[n]:
                    tx_PSD[k] = tx_PSD[k]-max(tx_PSD[k] + pathLoss[k] - thresh_Adj[n],0)
                elif tx_PSD[k]+pathLoss[k] < thresh_Adj[n]-1:
                    tx_PSD[k] = tx_PSD[k]+min(thresh_Adj[n]-1-tx_PSD[k]-pathLoss[k],37-tx_PSD[k])
            txPSD_Table.loc[len(txPSD_Table)]=tx_PSD - gain
            n+=1
            nIt+=1
            tx_PSD_array = np.array(tx_PSD, dtype=float)
            pathLoss_array = np.array(pathLoss, dtype=float)
            mask = ~np.isnan(tx_PSD_array)
            if self.pred_err_box.value:
                r_pathLoss = self.real_path_loss(nIt,pathLoss,mask,0,self.pred_err_value.value)
                # print(r_pathLoss)
                interf[n]=10*np.log10(np.sum(10**((tx_PSD + r_pathLoss)/10)))
            else:
                interf[n] = 10*np.log10(np.sum(10**((tx_PSD + pathLoss)/10)))
            if interf[n]> thresh_Target:
                thresh_Adj[n]=thresh_Adj[n-1]-1
            elif interf[n]<thresh_Target-1:
                thresh_Adj[n]=thresh_Adj[n-1]+1
            else:
                thresh_Adj[n]=thresh_Adj[n-1]
        #print(f"The loop ran {nIt} times.")
        return(nIt, thresh_Adj, interf, tx_PSD, txPSD_Table)
    
    # DSS Algorithm 2: Disallow transmit power increases
    def dss_disallow_increases(self, nIt, thresh_Target, gain, pathLoss, thresh_Adj, interf, tx_PSD, txPSD_Table):
        n=0
        # Adjust Transmit Powers to Meet Target Inteference Level
        while interf[n] > thresh_Target or (interf[n] < thresh_Target-1 and not np.all(self.tx_RadPSD==tx_PSD)):
            for k in range(self.num_Transmitters):
                if tx_PSD[k]+pathLoss[k] > thresh_Adj[n]:
                    tx_PSD[k] = tx_PSD[k]-max(tx_PSD[k] + pathLoss[k] - thresh_Adj[n],0)
                elif tx_PSD[k]+pathLoss[k] < thresh_Adj[n]-1:
                    tx_PSD[k] = tx_PSD[k]
            txPSD_Table.loc[len(txPSD_Table)]=tx_PSD - gain
            n+=1
            nIt+=1
            tx_PSD_array = np.array(tx_PSD, dtype=float)
            pathLoss_array = np.array(pathLoss, dtype=float)
            mask = ~np.isnan(tx_PSD_array)
            if self.pred_err_box.value:
                r_pathLoss = self.real_path_loss(nIt,pathLoss,mask,0,self.pred_err_value.value)
                # print(r_pathLoss)
                interf[n]=10*np.log10(np.sum(10**((tx_PSD + r_pathLoss)/10)))
            else:
                interf[n] = 10*np.log10(np.sum(10**((tx_PSD + pathLoss)/10)))
            if interf[n]> thresh_Target:
                thresh_Adj[n]=thresh_Adj[n-1]-1
            elif interf[n]<thresh_Target-1:
                thresh_Adj[n]=thresh_Adj[n-1]+1
            else:
                thresh_Adj[n]=thresh_Adj[n-1]
        #print(f"The loop ran {nIt} times.")
        return(nIt, thresh_Adj, interf, tx_PSD, txPSD_Table)
    
    # DSS Algorithm 3: Disable transmitters when transmit power drops below a threshold
    def dss_disable_below_thresh(self, nIt, thresh_Target, gain, pathLoss, thresh_Adj, interf, tx_PSD, txPSD_Table):
        n=0
        # Adjust Transmit Powers to Meet Target Inteference Level
        psd_ShutDown = self.tx_RadPSD + self.tx_dis_thresh.value
        # print(psd_ShutDown)
        while interf[n] > thresh_Target or (interf[n] < thresh_Target-1 and not np.all(np.isnan(tx_PSD) | (self.tx_RadPSD==tx_PSD))):
            # psd_ShutDown = self.tx_dis_thresh.value
            for k in range(self.num_Transmitters):
                if tx_PSD[k] <= psd_ShutDown[k]:
                    tx_PSD[k] = None
                elif tx_PSD[k]+pathLoss[k] > thresh_Adj[n]:
                    tx_PSD[k] = tx_PSD[k]-max(tx_PSD[k] + pathLoss[k] - thresh_Adj[n],0)
                elif tx_PSD[k]+pathLoss[k] < thresh_Adj[n]-1:
                    tx_PSD[k] = tx_PSD[k]+min(thresh_Adj[n]-1-tx_PSD[k]-pathLoss[k],self.tx_RadPSD[k]-tx_PSD[k])
            txPSD_Table.loc[len(txPSD_Table)]=tx_PSD - gain
            n+=1
            nIt+=1
            tx_PSD_array = np.array(tx_PSD, dtype=float)
            pathLoss_array = np.array(pathLoss, dtype=float)
            mask = ~np.isnan(tx_PSD_array)
            v_tx_PSD = tx_PSD_array[mask]
            v_pathLoss = pathLoss_array[mask]
            if self.pred_err_box.value:
                r_pathLoss = self.real_path_loss(nIt,v_pathLoss,mask,0,self.pred_err_value.value)
                # print(r_pathLoss)
                v_summation = np.nansum(10**((v_tx_PSD + r_pathLoss)/10))
            else:
                v_summation = np.nansum(10**((v_tx_PSD + v_pathLoss)/10))
            if v_summation > 0:
                interf[n]=10*np.log10(v_summation)
            else:
                interf[n] = float('-inf')
            # print(n, interf[n])
            if interf[n]> thresh_Target:
                thresh_Adj[n]=thresh_Adj[n-1]-1
            elif interf[n]<thresh_Target-1:
                thresh_Adj[n]=thresh_Adj[n-1]+1
            else:
                thresh_Adj[n]=thresh_Adj[n-1]
        # print(f"The loop ran {nIt} times.")
        return(nIt, thresh_Adj, interf, tx_PSD, txPSD_Table)
   
    # Nominal Interference Calculation
    def calc_clicked(self,b):   # Function to Calculate the Nominal Interference
        with self.algo:
            self.algo.clear_output()  # clear previous inteference info
        with self.calcs:
            self.calcs.clear_output()  # clear previous algorithm results

            # Intialize the Radar Protection Threshold
            self.thresh_Radar = radar_Inputs['cur_InitThresh']
            
            # Dimension Commercial Transmitters and Transmit Powers
            self.num_Transmitters = tx_Inputs['cur_NumTx']
            low_Power = tx_Inputs['cur_TxPSDlow']  # dBm/MHz
            high_Power = tx_Inputs['cur_TxPSDhigh']  # dBm/MHz
            
            # Interleave Low and High Power Transmitters in an Array
            tx_PSD = np.empty(self.num_Transmitters)
            for i in range(self.num_Transmitters):
                if i%2==0:
                    tx_PSD[i]=low_Power
                else:
                    tx_PSD[i]=high_Power
            
            # Compute Radiated PSD and EIRP using Channel Bandwidth and Antenna Gain            
            bw = tx_Inputs['cur_TxBW']  # MHz
            gain = tx_Inputs['cur_AntGain']  # dB
            tx_Bandwidths = np.linspace(bw,bw,self.num_Transmitters)
            tx_AntGain = np.linspace(gain,gain,self.num_Transmitters)
            self.tx_RadPSD = tx_PSD + tx_AntGain  # dBm/MHz
            self.tx_MaxPSD = np.linspace(37,37,self.num_Transmitters)   # dBm/MHz
            self.tx_EIRP = self.tx_RadPSD + 10*np.log10(tx_Bandwidths)  # dBm
            
            # Specify Path Loss per Link
            # This method assigns the lowest loss to the first two transmitters,
            # and then increases the loss by a specified amount for each successive
            # pair of transmitters.
            loss_Lowest = tx_Inputs['cur_InitPL']  # dB
            self.path_Loss = np.linspace(loss_Lowest,loss_Lowest,self.num_Transmitters)
            loss_adjust = 0
            loss_incr = -tx_Inputs['cur_DeltaPL']  # dB
            for i in range(self.num_Transmitters):
                if i%2==0:
                    self.path_Loss[i] = loss_Lowest + loss_adjust
                else:
                    self.path_Loss[i] = self.path_Loss[i-1]
                    loss_adjust = loss_adjust + loss_incr
            
            # Compute Received Signal Levels and Aggregate Interference
            self.rx_RSSI =  self.tx_EIRP + self.path_Loss  # dBm
            self.rx_PSDdB = self.tx_RadPSD + self.path_Loss  # dBm/MHz
            self.rx_PSDmW = 10**(self.rx_PSDdB/10)  # mW/MHz
            self.int_AggmW = self.rx_PSDmW.sum()  # mW/MHz
            self.int_AggdB = 10*np.log10(self.int_AggmW)   # dBm/MHz
            if self.int_AggdB > self.thresh_Radar:
                correction = self.thresh_Radar - self.int_AggdB   # dB
            else:
                correction = 0.0
            
            # Layout for Results
            label_AggInt_before = wg.Label(value="Aggregate Interference:")
            AggInt = wg.Label(value=str(round(self.int_AggdB,0)))
            label_AggInt_after = wg.Label(value="dBm/MHz")
            out_AggInt = wg.HBox([label_AggInt_before, AggInt, label_AggInt_after])
            label_Corr_before = wg.Label(value="Correction Needed:")
            Corr = wg.Label(value=str(round(correction,0)))
            label_Corr_after = wg.Label(value="dB")
            out_Corr = wg.HBox([label_Corr_before, Corr, label_Corr_after])
            grid3 = wg.GridspecLayout(3,2,width='650px')
            grid3[0,0] = wg.HTML(value="--------------------------------------------------------")
            grid3[1,0] = out_AggInt
            grid3[0,1] = wg.HTML(value="--------------------------------------------------------")
            grid3[1,1] = out_Corr
            display(grid3) 
            
            # Plot Individual Received Signal Levels
            desired_width = 5 # inches
            aspect_ratio_1 = (10/self.num_Transmitters)*1.5
            dynamic_height = desired_width/aspect_ratio_1
            self.tx_Labels = [f'Tx{i+1}' for i in range(self.num_Transmitters)]
            if self.int_AggdB<=self.thresh_Radar:
                color_AggInt = 'green'
            else:
                color_AggInt = 'red'
            plt.figure(figsize=(desired_width,dynamic_height))
            plt.rc('xtick',labelsize=8)
            plt.rc('ytick',labelsize=8)
            #plt.barh(y=tx_Labels,width=rx_PSDdB) # Bar graph alternative to scatter plot
            y_positions = range(self.num_Transmitters)
            plt.scatter(self.rx_PSDdB,y_positions,s=60,color='blue',alpha=0.7)
            plt.yticks(y_positions,self.tx_Labels)
            plt.axvline(x=self.thresh_Radar,color='darkslategray',linestyle=':',linewidth=1,label=f'Threshold= {self.thresh_Radar}')
            plt.axvline(x=self.int_AggdB,color=color_AggInt,linestyle=':',linewidth=1,label=f'Aggregate= {int(self.int_AggdB.round(0))}')
            plt.xlim(right=-110)
            plt.xlabel('Received Signal Level (dBm/MHz)',fontsize=8)
            plt.ylabel('Transmitters',fontsize=10)
            plt.title('Predicted Signal Levels at the Radar',fontsize=10)
            plt.gca().invert_yaxis()
            #plt.gca().invert_xaxis()  # Option to reverse x-axis values
            plt.legend(loc='upper left',bbox_to_anchor=(1,1),fontsize=8)
            plt.show()
    
    # Execute DSS Algorithm
    def run_clicked(self,b):
        with self.algo:
            self.algo.clear_output()  # clear previous algorithm results
            if self.int_AggdB is None:
                display(wg.HTML("<font color='red'>Please calculate interference first!</font>"))
                return

            # Initialize Counters and Arrays
            nIt=0
            interf = np.zeros(100)
            interf[0] = self.int_AggdB
            thresh_Target = self.thresh_Radar
            thresh_Adj = np.zeros(100)
            thresh_Adj[0] = thresh_Target
            txLabels = self.tx_Labels.copy()
            tx_PSD = self.tx_RadPSD.copy()
            rx_PSD = self.rx_PSDdB.copy()
            pathLoss = self.path_Loss.copy()

            # Create Table of Adapted Tx Powers
            tx_Columns = [f'Tx{i+1}' for i in range(self.num_Transmitters)]
            txPSD_Table = pd.DataFrame(columns=tx_Columns)
            txPSD_Table.loc[0]=tx_PSD - tx_Inputs['cur_AntGain']
            
            # Adjust Transmit Powers to Meet Target Inteference Level
            if self.algo_Option == 1:
                nIt, thresh_Adj, interf, tx_PSD, txPSD_Table = self.dss_converge_rx(
                    nIt, thresh_Target, tx_Inputs['cur_AntGain'], pathLoss,
                    thresh_Adj, interf, tx_PSD, txPSD_Table
                )
                num_Iterations = nIt
            elif self.algo_Option == 2:
                nIt, thresh_Adj, interf, tx_PSD, txPSD_Table = self.dss_disallow_increases(
                    nIt, thresh_Target, tx_Inputs['cur_AntGain'], pathLoss,
                    thresh_Adj, interf, tx_PSD, txPSD_Table
                )
                num_Iterations = nIt
            elif self.algo_Option == 3:
                nIt, thresh_Adj, interf, tx_PSD, txPSD_Table = self.dss_disable_below_thresh(
                    nIt, thresh_Target, tx_Inputs['cur_AntGain'], pathLoss,
                    thresh_Adj, interf, tx_PSD, txPSD_Table
                )
                num_Iterations = nIt
            elif self.algo_Option == 4:
                with self.algo:
                    display(wg.VBox(
                        [wg.HTML(value="--------------------------------------------------------"),
                         wg.HTML(value="<b style='font-size: 14px;'>Summary of Algorithm Results</b>")
                        ])
                           )
                    compare_Table, interf_Adapt = self.compare()
                    display(compare_Table)
                    display(wg.HTML(value="-----"))
                    nIt_Max = compare_Table.loc['Number of Iterations'].max()
                    interf_Adapt_NZ = interf_Adapt.replace(0, np.nan)
                    desired_height = 4 # inches
                    dynamic_width = min(6 + (0.25*nIt_Max/10),8)
                    fig,ax = plt.subplots(figsize=(dynamic_width,desired_height))
                    interf_Adapt_NZ.plot(ax=ax)
                    plt.axhline(y=thresh_Target,color='green',linestyle=':',linewidth=1,label=f'Target= {thresh_Target}')
                    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
                    ax.set_title('Interference at Radar as Transmitters Adapt',fontsize=10)
                    ax.set_xlabel('Iteration Number',fontsize=8)
                    ax.set_ylabel('Aggregate Interference (dBm/MHz)',fontsize=8)
                    ax.legend(loc='upper right',fontsize=8)
                    ax.tick_params(axis='both', which='major', labelsize=8)
                    plt.show()
                    return
            else:
                print("SOMETHING WENT WRONG IN THE SELECTION OF AN ALGORITHM")
                return

            # Layout Results of Adaptation Algorithm
            label_Algo = wg.HTML(value="DSS Algorithm: ")
            algo_Name = wg.Label(value=str(self.algo_Options[self.algo_Option-1]))
            out_Algo = wg.HBox([label_Algo, algo_Name])
            label_InitAggInt_before = wg.Label(value="Initial Aggregate Interference:")
            InitAggInt = wg.Label(value=str(round(interf[0])))
            label_InitAggInt_after = wg.Label(value="dBm/MHz")
            out_InitAggInt = wg.HBox([label_InitAggInt_before, InitAggInt, label_InitAggInt_after])
            label_numIterations = wg.Label(value="Number of Iterations:")
            NumIterations = wg.Label(value=str(num_Iterations))
            out_numIterations = wg.HBox([label_numIterations, NumIterations])
            label_FinalAggInt_before = wg.Label(value="Final Aggregate Interference:")
            FinalAggInt = wg.Label(value=str(round(interf[num_Iterations])))
            label_FinalAggInt_after = wg.Label(value="dBm/MHz")
            out_FinalAggInt = wg.HBox([label_FinalAggInt_before, FinalAggInt, label_FinalAggInt_after])
            grid4 = wg.GridspecLayout(4,2,width='650px')
            grid4[0,0] = wg.HTML(value="--------------------------------------------------------")
            grid4[1,0] = out_Algo
            grid4[2,0] = out_numIterations
            grid4[0,1] = wg.HTML(value="--------------------------------------------------------")
            grid4[1,1] = out_InitAggInt
            grid4[2,1] = out_FinalAggInt
            display(grid4)
            
            # Plot Aggregate Interference vs. Iteration
            desired_height = 4 # inches
            dynamic_width = min(6 + (0.25*num_Iterations/10),8)
            plt.figure(figsize=(dynamic_width,desired_height))
            plt.rc('xtick',labelsize=8)
            plt.rc('ytick',labelsize=8)
            x_positions=list(range(0,num_Iterations+1))
            plt.scatter(x_positions,interf[0:num_Iterations+1],s=30)
            plt.plot(x_positions,interf[0:num_Iterations+1],linestyle='-',linewidth=1)
            plt.axhline(y=thresh_Target,color='green',linestyle=':',linewidth=1,label=f'Target= {thresh_Target}')
            plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
            plt.xlabel('Iteration Number',fontsize=8)
            plt.ylabel('Aggregate Interference (dBm/MHz)',fontsize=8)
            plt.title('Interference at Radar as Transmitters Adapt',fontsize=10)
            plt.show()
            
            # Adjusted Transmitter Power Levels
            tx_PSDorig = self.tx_RadPSD - tx_Inputs['cur_AntGain']
            tx_PSDadj = tx_PSD - tx_Inputs['cur_AntGain']
            atten = tx_PSD - self.tx_RadPSD
            rx_PSDadj = tx_PSD + pathLoss
            df = pd.DataFrame({
                'Orig Tx PSD':tx_PSDorig,
                'Adj. Tx PSD':tx_PSDadj,
                'Attenuation':atten,
                'Final Rx PSD':rx_PSDadj
            })
            df.index=txLabels
            tx_Table = wg.Output()
            with tx_Table:
                display(df)
            display(tx_Table)
            
            # Plot Results of Each Iteration
            desired_height = 4 # inches
            dynamic_width = min(6 + (0.25*num_Iterations/10),8)
            ax = txPSD_Table.plot(figsize=(dynamic_width,desired_height))
            plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
            plt.xlabel('Iteration Number',fontsize=8)
            plt.ylabel('Tx PSD (dBm/MHz)',fontsize=8)
            plt.legend(title='Series')
            plt.legend(loc='upper left',bbox_to_anchor=(1,1),fontsize=8)
            plt.title('Transmitter PSD Adaptation per Iteration',fontsize=10)
            for column in txPSD_Table.columns:
                x=txPSD_Table.index[-1]
                y=txPSD_Table[column].iloc[-1]
                i=txPSD_Table.columns.get_loc(column)
                # print(x,y)
                ax.annotate(
                    text=column,
                    xy=(x,y),
                    xytext=(-12,-6+12*(i%2)),
                    textcoords='offset points',
                    fontsize=8
                )
            plt.show()
            
            tx_Adaptation = wg.Output()
            with tx_Adaptation:
                display(self.txPSD_Table)
                display(self.tx_Adaptation)

    # Compare All Algorithms
    def compare(self):               
        # Initialize Counters and Arrays
        nIt=0
        interf = np.zeros(100)
        interf[0] = self.int_AggdB
        thresh_Target = self.thresh_Radar
        thresh_Adj = np.zeros(100)
        thresh_Adj[0] = thresh_Target
        txLabels = self.tx_Labels.copy()
        tx_PSD = self.tx_RadPSD.copy()
        rx_PSD = self.rx_PSDdB.copy()
        pathLoss = self.path_Loss.copy()
        
        # Create Table of Adapted Tx Powers
        tx_Columns = [f'Tx{i+1}' for i in range(self.num_Transmitters)]
        txPSD_Table = pd.DataFrame(columns=tx_Columns)
        txPSD_Table.loc[0]=tx_PSD - tx_Inputs['cur_AntGain']

        # Create Random Number Arrays
        rng = np.random.default_rng()
        iterations_max = 1000
        if self.pred_err_box.value:
            self.arr_Random = [rng.normal(loc=0,scale=self.pred_err_value.value,size=self.num_Transmitters) for _ in range(iterations_max)]
        else:
            self.arr_Random = [np.zeros(self.num_Transmitters) for _ in range(iterations_max)]
            
        # Create Table for Aggregate Interference Adaptation
        scenarios = ['Converge','Disallow Increase','Disable Below Thresh']
        interf_Adapt = pd.DataFrame(columns=scenarios)
        
        # Create Table for Comparison Values
        metrics = ['Number of Iterations',
                   'Aggregate Interference (dBm/MHz)',
                   'Transmitters Impaired (dtx+atten)',
                   'Transmitters Disabled',
                   'Transmitters Attenuated',
                   'Active Tx w/ Attenuation > 3 dB',
                   'Active Tx w/ Attenuation > 10 dB',
                   'Average Rx Level per Tx (dBm/MHz)'
                  ]
        compare_Table = pd.DataFrame(columns=scenarios,index=metrics,data=np.nan) 
        
        # Run the First Algorithm: Converge 
        nIt, thresh_Adj, interf, tx_PSD, txPSD_Table = self.dss_converge_rx(
            nIt, thresh_Target, tx_Inputs['cur_AntGain'], pathLoss,
            thresh_Adj, interf, tx_PSD, txPSD_Table
        )
        tx_PSD_array = np.array(tx_PSD, dtype=float)
        pathLoss_array = np.array(pathLoss, dtype=float)
        mask = ~np.isnan(tx_PSD_array)
        v_tx_PSD = tx_PSD_array[mask]
        v_pathLoss = pathLoss_array[mask]
        tx_PSDorig = self.tx_RadPSD - tx_Inputs['cur_AntGain']
        tx_PSDadj = tx_PSD - tx_Inputs['cur_AntGain']
        atten = tx_PSD - self.tx_RadPSD
        rx_PSDadj = tx_PSD + pathLoss
        # rx_avg = np.mean(rx_PSDadj)
        rx_avg = 10*np.log10(np.mean(10**(rx_PSDadj/10)))
        interf_Adapt['Converge'] = interf
        compare_Table['Converge'] = [
            nIt,
            str(round(interf[nIt])),
            f"{(np.sum(np.isnan(tx_PSD_array)==True) + np.sum(atten<0))/self.num_Transmitters:.0%}",
            f"{np.sum(np.isnan(tx_PSD_array)==True)/self.num_Transmitters: .0%}",
            f"{np.sum(atten<0)/self.num_Transmitters: .0%}",
            f"{np.sum(atten<-3)/len(v_tx_PSD): .0%}",
            f"{np.sum(atten<-10)/len(v_tx_PSD): .0%}",
            str(round(rx_avg))
        ]

        # Reset Counters and Arrays
        nIt=0
        interf = np.zeros(100)
        interf[0] = self.int_AggdB
        thresh_Target = self.thresh_Radar
        thresh_Adj = np.zeros(100)
        thresh_Adj[0] = thresh_Target
        txLabels = self.tx_Labels.copy()
        tx_PSD = self.tx_RadPSD.copy()
        rx_PSD = self.rx_PSDdB.copy()
        pathLoss = self.path_Loss.copy()

        # Run the Second Algorithm: Disallow Increases 
        nIt, thresh_Adj, interf, tx_PSD, txPSD_Table = self.dss_disallow_increases(
            nIt, thresh_Target, tx_Inputs['cur_AntGain'], pathLoss,
            thresh_Adj, interf, tx_PSD, txPSD_Table
        )
        tx_PSD_array = np.array(tx_PSD, dtype=float)
        pathLoss_array = np.array(pathLoss, dtype=float)
        mask = ~np.isnan(tx_PSD_array)
        v_tx_PSD = tx_PSD_array[mask]
        v_pathLoss = pathLoss_array[mask]
        tx_PSDorig = self.tx_RadPSD - tx_Inputs['cur_AntGain']
        tx_PSDadj = tx_PSD - tx_Inputs['cur_AntGain']
        atten = tx_PSD - self.tx_RadPSD
        rx_PSDadj = tx_PSD + pathLoss
        # rx_avg = np.mean(rx_PSDadj)
        rx_avg = 10*np.log10(np.mean(10**(rx_PSDadj/10)))
        interf_Adapt['Disallow Increase'] = interf
        compare_Table['Disallow Increase'] = [
            nIt,
            str(round(interf[nIt])),
            f"{(np.sum(np.isnan(tx_PSD_array)==True) + np.sum(atten<0))/self.num_Transmitters:.0%}",
            f"{np.sum(np.isnan(tx_PSD_array)==True)/self.num_Transmitters: .0%}",
            f"{np.sum(atten<0)/self.num_Transmitters: .0%}",
            f"{np.sum(atten<-3)/len(v_tx_PSD): .0%}",
            f"{np.sum(atten<-10)/len(v_tx_PSD): .0%}",
            str(round(rx_avg))
        ]

        # Reset Counters and Arrays
        nIt=0
        interf = np.zeros(100)
        interf[0] = self.int_AggdB
        thresh_Target = self.thresh_Radar
        thresh_Adj = np.zeros(100)
        thresh_Adj[0] = thresh_Target
        txLabels = self.tx_Labels.copy()
        tx_PSD = self.tx_RadPSD.copy()
        rx_PSD = self.rx_PSDdB.copy()
        pathLoss = self.path_Loss.copy()
        
        # Run the Third Algorithm: Disable Below Threshold
        nIt, thresh_Adj, interf, tx_PSD, txPSD_Table = self.dss_disable_below_thresh(
            nIt, thresh_Target, tx_Inputs['cur_AntGain'], pathLoss,
            thresh_Adj, interf, tx_PSD, txPSD_Table
        )
        tx_PSD_array = np.array(tx_PSD, dtype=float)
        pathLoss_array = np.array(pathLoss, dtype=float)
        mask = ~np.isnan(tx_PSD_array)
        v_tx_PSD = tx_PSD_array[mask]
        v_pathLoss = pathLoss_array[mask]
        tx_PSDorig = self.tx_RadPSD - tx_Inputs['cur_AntGain']
        tx_PSDadj = v_tx_PSD - tx_Inputs['cur_AntGain']
        atten = tx_PSD - self.tx_RadPSD
        rx_PSDadj = tx_PSD + pathLoss
        # rx_avg = np.mean(rx_PSDadj[mask])
        rx_avg = 10*np.log10(np.mean(10**(rx_PSDadj[mask]/10)))
        interf_Adapt['Disable Below Thresh'] = interf
        compare_Table['Disable Below Thresh'] = [
            nIt,
            str(round(interf[nIt])),
            f"{(np.sum(np.isnan(tx_PSD_array)==True) + np.sum(atten<0))/self.num_Transmitters:.0%}",
            f"{np.sum(np.isnan(tx_PSD_array)==True)/self.num_Transmitters: .0%}",
            f"{np.sum(atten<0)/self.num_Transmitters: .0%}",
            f"{np.sum(atten<-3)/len(v_tx_PSD): .0%}",
            f"{np.sum(atten<-10)/len(v_tx_PSD): .0%}",
            str(round(rx_avg))
        ]
        return(compare_Table, interf_Adapt)


# Instantiate the Class
Computations = InterferenceMitigation()


Output()

Output()

## Print to PDF
If you would like to export a copy of this anlaysis as a PDF, click the _Create PDF_ button below.<br>
Note that only the current view will be captured.

In [6]:
# import subprocess
# from datetime import datetime
# import os

# current_date = datetime.now().strftime("%Y%m%d")

# notebook_path = r"C:\Users\jattan000\OneDrive - Comcast\Tools\JupyterLab\DSS_Algorithm.ipynb"
# output_dir = r"C:\Users\jattan000\OneDrive - Comcast\Tools\JupyterLab\reports"
# base_filename = f"DSS_Report_{current_date}"

# os.makedirs(output_folder, exist_ok=True)

# temp_html = os.path.join(output_dir, f"{base_filename}_temp.html")

# subprocess.run([
#     "jupyter", "nbconvert",
#     "--to", "html", "--execute", "--no-input", "--allow-errors", notebook_path,
#     "--output", temp_html
# ], check=True)

# interactive_html = os.path.join(output_dir, f"{base_filename}_interactive.html")

# subprocess.run([
#     "nbinteract",
#     temp_html,
#     "-o", interactive_html
# ], check=True)